# A premier league table projection

### Due to corona virus concerns, the premier league has been suspended indefinitely. As a result, there has been a lot of confusion on whether Liverpool should be awarded the title, which teams quality for Europe and which teams get relegated

### With current points total as base, I projected their points across 38 games. I used fixture difficulty rating stats and fixture list from Fantasy PL. 

* Unsurprisingly, Liverpool win the title.
* Bournemouth, Aston Villa and Norwich are relegated.
* Chelsea, Leicester, Man City(*) and Liverpool qualify for Champions League.

###### (if City win the case against UEFA)

#### Biggest issues with my model

* teams with same points total should be ranked in descending order of their goal difference. I did not consider goals scored and concended parameter.

* form is a huge factor. Just because a team has high quality players doesn't mean they are always the favourites. Form matters.

* injuries. Injuries to key players can hamper whole seasons. Spurs this season is a good example

### Going forward, how I plan on tackling these key issues

* Based on average goals scored per game and average goals conceded per game, we can get to know about a team's offensive and defensive rating. Liverpool for eg, are a very good defensive side. Whereas City are very good offensively. Based on these two stats, we can predict the margin of victory which will ultimately give us goal difference.

* ~~Keeping a list of form over last 5 fixtures will help take form of the team into account.~~ (fixed in this version)

* based on the number of injuries and importance of the player, team's difficulty rating can be adjusted.


### Dataframes

* fdr - Fixture difficulty rating. Based on team's quality, they are ranked from 5 to 2, 5 being hardest.
* fix - Remaining fixture list for all teams
* table - Present day premier league standings
* form - form over past 5 games

### Approach

* for every gameweek, each team's match with their opponent in that gameweek is considered. So outer loop is remaining gameweek list and inner loop is the team list.
* fdr rankings are used to find out which is the better team.
* extra weights are given for teams playing at home.
* these win/loss/draw probability values are calculated for every match and given to a random.choice function
* based on the outcome, points are assigned to the team using a dictionary. 3 for a win, 0 for a defeat and 1 for a draw.
* after all the match results are found out, individual team's points across gameweeks are summed and added to their initial total to find out end of season points.

#### We now consider form over last 5 matches and give that a weight while making prediction

In [53]:
import pandas as pd
fdr = pd.read_csv("D:\\Praxis\\ML\\premier_league_dataset\\FDR.csv")
fix = pd.read_csv("D:\\Praxis\\ML\\premier_league_dataset\\Fixtures.csv")
table = pd.read_csv("D:\\Praxis\\ML\\premier_league_dataset\\table.csv")
rem = pd.read_csv("D:\\Praxis\\ML\\premier_league_dataset\\remaining.csv")
table_points = table[['Position','Team','Pts']]
df=pd.read_csv("D:\\Praxis\\ML\\premier_league_dataset\\form.csv", sep=',',header=None)
form = df.values
table_points = table_points.dropna()
from numpy.random import choice
import numpy as np

In [54]:
gw = 31
points = {"Win":3, "Draw": 1, "Lose":0}
count = 0
n = 20
while count<10:    
    gw = str(gw)
    for i in table_points['Team']:
        X0 = np.ones((n,1))
        form = np.hstack((form,X0))
        gw = str(gw)
        our_team = i
        match = fix.loc[(fix.Team == our_team), gw]
        match1 = match.values[0]
        #our_team = 'Liverpool'
        our_value = fdr[fdr.Team == our_team].DifficultyRating

        if match.values[0] != "BLANK":
        #print("enter")
            #count = count + 1
            match1 = (match1.split())
            oppo = match1[0]
            home_or_away = match1[1].replace('(','').replace(')','')            
            opp_value = fdr[fdr.Abbr == oppo].DifficultyRating
            opp_value

            diff_in_quality = our_value.values[0] - opp_value.values[0]
            diff_in_quality

            outcomes = ["Win", "Draw", "Lose"]

            win_chance = 0.25 * diff_in_quality
            if win_chance == 0:
                draw = 0.50
                win = 0.25
                lose = 0.25
            elif diff_in_quality > 0:
                win = 0.50
                draw = 0.25
                lose = 0.25
            else:
                win = 0.25
                draw = 0.25
                lose = 0.50
            
            if home_or_away == 'H':
                win = win + 0.10
                lose = lose - 0.10
            
            else:
                win = win - 0.10
                lose = lose + 0.10
            
            opponent = rem[rem.Abbr == oppo].Team
            xx = np.where(form == opponent.values)
            n,m = form.shape
            opp_form_sum = np.sum(form[xx[0],m-6:m-1])
            
            yy = np.where(form == our_team)
            n,m = form.shape
            our_form_sum = np.sum(form[xx[0],m-6:m-1])
            
            if(our_form_sum-opp_form_sum)>5:
                win = win + 0.10
                lose = lose - 0.10
            elif(opp_form_sum - our_form_sum)>5:
                win = win - 0.10
                lose = lose + 0.10
                
            probabilities = [win,draw,lose]

            outcome = (choice(outcomes, p=probabilities))

            #total_points = total_points+ points[outcome]
            gw = str(gw)
            match1 = rem.loc[(rem.Abbr == oppo), gw]
            if match1.values[0] == 'NOTPLAYED':
                #match1.values[0] = 3
                if outcome == 'Win':
                    x = 0
                elif outcome == 'Lose':
                    x = 3
                else:
                    x = 1
                rem.loc[(rem.Abbr == oppo), gw] = x
                form[xx[0],m-1] = x
            our_team_abbr = fdr[fdr.Team == our_team].Abbr

            match2 = rem.loc[(rem.Abbr == our_team_abbr.values[0]), gw]
            if match2.values[0] == 'NOTPLAYED':
                #match2.values[0] = 3
                rem.loc[(rem.Abbr == our_team_abbr.values[0]), gw] = points[outcome]
                form[yy[0],m-1] = points[outcome]

            gw = int(gw)
    gw = int(gw)
    gw = gw + 1
    count = count + 1


In [55]:
for i in table_points['Team']:
    our_team = i
    our_points = table_points[table_points['Team']==our_team].Pts
    total_points = our_points.values[0]
    count = 0
    gw = 31
    while(count < 10):
            gw = str(gw)
            match = rem.loc[(rem.Team == our_team), gw]
            if match.values[0] != "BLANK":
                total_points = total_points + int(match.values[0])
            count = count + 1
            gw = int(gw)
            gw = gw + 1
    rem.loc[rem['Team'] == our_team, 'Current_points'] = table_points[table_points['Team']==our_team].Pts
    rem.loc[rem['Team'] == our_team, 'Projected_Points'] = total_points
sorted_rem = rem.sort_values(by = ['Projected_Points'], ascending = False)
sorted_rem

,Team,Abbr,31,32,33,34,35,36,37,38,39,40,Current_points,Projected_Points
0,Liverpool,LIV,3,1,3,3,3,3,1,0,BLANK,BLANK,82.0,99.0
2,Leicester,LEI,BLANK,1,3,3,0,3,1,1,0,BLANK,53.0,65.0
1,Man City,MCI,BLANK,1,0,BLANK,0,1,3,1,1,0,57.0,64.0
3,Chelsea,CHE,BLANK,0,3,3,3,1,1,1,1,BLANK,48.0,61.0
6,Sheffield United,SHU,BLANK,1,3,3,0,0,3,3,1,0,43.0,57.0
4,Man United,MUN,BLANK,0,0,1,0,3,3,1,1,BLANK,45.0,54.0
7,Tottenham,TOT,1,1,1,1,1,1,1,3,BLANK,BLANK,41.0,51.0
8,Arsenal,ARS,BLANK,3,0,0,1,0,3,3,1,BLANK,40.0,51.0
5,Wolves,WOL,1,0,3,0,1,1,0,1,BLANK,BLANK,43.0,50.0
9,Burnley,BUR,3,0,0,1,0,1,3,3,BLANK,BLANK,39.0,50.0
